# Table of Contents
 <p><div class="lev1 toc-item"><a href="#setup" data-toc-modified-id="setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>setup</a></div><div class="lev1 toc-item"><a href="#auxilliary-RMG-methods" data-toc-modified-id="auxilliary-RMG-methods-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>auxilliary RMG methods</a></div><div class="lev1 toc-item"><a href="#make-species-block" data-toc-modified-id="make-species-block-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>make species block</a></div><div class="lev1 toc-item"><a href="#make-reaction-block" data-toc-modified-id="make-reaction-block-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>make reaction block</a></div>

This notebook creates sections of a cantera input file with RMG naming convention, since it is more intuitive than Chemkin's limited naming system. 

This notebook reads in a chemkin input file and species dictionary. It then uses RMG to label species, which is used throughout the conversion. It creates cantera objects with RMG's `toCantera` method, but since they cannot be easily converted into the input file string, it then creates similar sounding objects from the ck2cti file, which then uses the `to_cti` method to obtain the input file syntax.

Currently, this notebook doesn't output a complete file. It gives you the species block, the thermo block and the kinetics block. You can then copy and paste them into your .cti file. It only supports Arrhenius kinetics and NASA polynomials.

Ideally, I'd like to see cantera utilize the __repr__ function to enable outputting the proper syntax, so there would be no need to utilize the ck2cti objects. In addition there could be a method, `toCti` in the solution object class in cantera. That would be for a later date.


# setup

In [1]:
from rmgpy.chemkin import loadChemkinFile
import os.path
import cantera as ct
from IPython.display import display

# locally hosted
# import ck2cti
from cantera.ck2cti import Species, Reaction, Arrhenius, NASA, MultiNASA, Parser, ENERGY_UNITS, QUANTITY_UNITS

In [2]:
mainPath = '/home/mark/research/isotopes/data/rmg_model_generation/20170627-ethanol-2C-1O/rmg/chemkin'

In [3]:
chemkinPath = os.path.join(mainPath, 'chem_annotated.inp')
speciesDictPath = os.path.join(mainPath,'species_dictionary.txt')

In [4]:
rmg_species, rmg_reactions = loadChemkinFile(chemkinPath, speciesDictPath, readComments = False, useChemkinNames=False)

# auxilliary RMG methods

In [5]:
def make_string_labels_independent(species):
    """
    This method reads in species labels and makes sure none of them conflict
    If a conflict occurs, the second occurance will have '-2' added
    """
    
    labels = ()
    for spec in species:
        duplicate_index = 1
        potential_label = spec.label
        while potential_label not in labels:
            duplicate_index += 1
            potential_label = spec.label + '-{}'.format(duplicate_index)
        spec.label = potential_label
        
#def make_reaction_label(rxn):
#    """
#    sets the label attribute of reaction to be 
#    """

In [6]:
parser = Parser()
parser.energy_units = 'J/kmol'
parser.output_energy_units = 'kcal/mol'
parser.quantity_units = 'kmol'
parser.output_quantity_units = 'mol'
parser.elements = "H D T C Ci O Oi N Ne Ar He Si S Cl".split(' ')

# make species block

In [7]:
# convert species to cti file string
ct_species = []
for spec in rmg_species:
    ct_spec = Species(label = spec.label)
    
    # get thermo
    multinasa = MultiNASA(Tmin = [spec.thermo.Tmin],
                         Tmax = [spec.thermo.Tmax],
                         comment = spec.thermo.comment)
    for polynomial in spec.thermo.polynomials:
        multinasa.polynomials.append(NASA(polynomial.coeffs, 
                     Tmin = [polynomial.Tmin.value_si], 
                     Tmax = [polynomial.Tmax.value_si],))
    ct_spec.thermo = multinasa
    
    # composition
    cantera_spec = spec.toCantera()
    ct_spec.composition = cantera_spec.composition
    ct_species.append(ct_spec)
    

In [8]:
species_block = ''
for spec in ct_species:
    species_block += spec.to_cti() + '\n'

In [9]:
print(species_block)

species(name='He',
        atoms='He:1.0',
        thermo=(NASA([200.00, 1000.00],
                     [ 2.50000000E+00,  0.00000000E+00,  0.00000000E+00,
                       0.00000000E+00,  0.00000000E+00, -7.45375000E+02,
                       9.28724000E-01]),
                NASA([1000.00, 6000.00],
                     [ 2.50000000E+00,  0.00000000E+00,  0.00000000E+00,
                       0.00000000E+00,  0.00000000E+00, -7.45375000E+02,
                       9.28724000E-01])))

species(name='Ar',
        atoms='Ar:1.0',
        thermo=(NASA([200.00, 1000.00],
                     [ 2.50000000E+00,  0.00000000E+00,  0.00000000E+00,
                       0.00000000E+00,  0.00000000E+00, -7.45375000E+02,
                       4.37967000E+00]),
                NASA([1000.00, 6000.00],
                     [ 2.50000000E+00,  0.00000000E+00,  0.00000000E+00,
                       0.00000000E+00,  0.00000000E+00, -7.45375000E+02,
                       4.37967000E+00])))



In [10]:
print(parser.getSpeciesString(ct_species,21))

He         Ar         Ne         N2         ethanol
                     C=C        O          [CH2]O     [CH3]      C[CH2]
                     C[CH]O     [H]        [CH]=C     C#C        [H][H]
                     C=O        C[O]       C          [CH]=O     [C-]#[O+]


# make reaction block

currently only works for arrhenius

In [11]:
rxn_block = ''
for rxn in rmg_reactions:
    ct_rxn = rxn.toCantera(useChemkinIdentifier=False)
    
    ck2cti_rxn = Reaction(index = rxn.index,
                         reactants = [(stoich, spec) for spec, stoich in ct_rxn.reactants.items()],
                         products = [(stoich, spec) for spec, stoich in ct_rxn.products.items()],
                         reversible = ct_rxn.reversible,
                         duplicate = ct_rxn.duplicate,
                         #ID = rxn.index,
                         )
    ck2cti_rxn.comment = rxn.kinetics.comment,

    # add kinetics
    # convert from cantera's kmol, J, s, m units to the input files' mol, cm, s, kcal units:
    rxn_order = sum([value for value in ct_rxn.reactants.values()])
    A_converted = ct_rxn.rate.pre_exponential_factor * (1e3 ** (rxn_order - 1))
    Ea_converted = ct_rxn.rate.activation_energy / 4.184e6
    ck2cti_rxn.kinetics = Arrhenius(A = (A_converted,QUANTITY_UNITS['MOL']), 
                                    b = ct_rxn.rate.temperature_exponent, 
                                    Ea = (Ea_converted ,ENERGY_UNITS['KCAL']),
                                    parser = parser)
    rxn_block += ck2cti_rxn.to_cti()
    rxn_block += '\n\n'

In [12]:
print rxn_block

reaction('C=C + O <=> ethanol', [5.880000e+02, 2.94, 53.1])

reaction('[CH]=C + [H] <=> C=C', [1.210000e+14, 0.0, 0.0])

reaction('[H] + C#C <=> [CH]=C', [1.030000e+09, 1.64, 2.11])

reaction('2.0 [CH]=C <=> C=C + C#C', [7.332204e+06, 1.885, -1.12])

reaction('[CH]=C + C[CH2] <=> 2.0 C=C', [4.560000e+14, -0.7, 0.0])

reaction('C=C + [H] <=> C[CH2]', [4.620000e+08, 1.64, 1.01])

reaction('[H][H] + [CH]=C <=> C=C + [H]', [2.362818e+01, 3.243, 3.347])

reaction('[CH]=C + [H] <=> [H][H] + C#C', [6.788225e+08, 1.5, -0.89])

reaction('2.0 [H] <=> [H][H]', [5.450000e+10, 0.0, 1.5])

reaction('[H] + C[CH2] <=> [H][H] + C=C', [1.083000e+13, 0.0, 0.0])

reaction('[CH3] + [CH2]O <=> ethanol', [3.542033e+14, -0.441, -0.055])

reaction('C=O + [H] <=> [CH2]O', [2.330000e+03, 3.17, 6.7])

reaction('[CH]=C + [CH2]O <=> C=O + C=C', [3.010000e+13, 0.0, 0.0])

reaction('[H] + [CH2]O <=> C=O + [H][H]', [2.000000e+13, 0.0, 0.0])

reaction('C[CH]O + [H] <=> ethanol', [2.268894e+13, 0.06, 0.041])

reaction('